# 01- Extracción de vulnerabilidades desde la API del NIST
Este notebook consulta la API pública del NIST para obtener vulnerabilidades (CVEs) que contengan una palabra clave específica (como 'windows') y que hayan sido modificadas en un año determinado. Luego exporta los datos filtrados a un archivo CSV para análisis posterior.

# Instalar dependencias necesarias en el terminal del DevContainer

In [5]:
pip install requests pandas matplotlib seaborn psycopg2-binary python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached matplotlib-3.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 43.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 51.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 50.7 MB/s eta 0:00:00:00

## Importacion de librerias necesarias y api key de NIST para no tener los bloqueos

In [6]:
import requests
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv
import os

# Cargar variables desde .env
load_dotenv()
API_KEY = os.getenv("NIST_API_KEY")

# Definir headers para autorización y agente
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; MyScript/1.0)",
    "apiKey": API_KEY
}


In [7]:
print(" API Key cargada correctamente:", API_KEY[:6], "...")
# Definir headers para autorización y agente
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; MyScript/1.0)",
    "apiKey": API_KEY
}

 API Key cargada correctamente: 2500f2 ...


## Variables modificables para otro tipo de consultas, podemos cambiar por lo que deseemos consultar

In [11]:
PALABRA_CLAVE = "windows"
ANIO_OBJETIVO = 2024
MAX_VULNS = 6000
ARCHIVO_SALIDA = "vulnerabilidades_2024.csv"

Endpoint de la API del NIST

In [9]:
url = "https://services.nvd.nist.gov/rest/json/cves/2.0"
resultados_filtrados = []
start_index = 0
page_size = 2000

Paginación de resultados para obtener múltiples páginas

In [10]:
while len(resultados_filtrados) < MAX_VULNS:
    params = {
        "resultsPerPage": page_size,
        "startIndex": start_index
    }

    #  AQUÍ incluimos los headers con la API key
    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:
        print(f" Error: {response.status_code}")
        print(response.text)
        break

    data = response.json()
    cves = data.get("vulnerabilities", [])
    if not cves:
        break

    for item in cves:
        try:
            cve = item["cve"]
            description = cve["descriptions"][0]["value"].lower()
            mod_date = cve["lastModified"]
            pub_date = cve["published"]
            source = cve.get("sourceIdentifier", "")
            metrics = cve.get("metrics", {})

            # Convertir fecha de modificación a año
            try:
                year = datetime.strptime(mod_date, "%Y-%m-%dT%H:%M:%S.%f").year
            except ValueError:
                year = datetime.strptime(mod_date, "%Y-%m-%dT%H:%M:%S").year

            if PALABRA_CLAVE.lower() in description and year == ANIO_OBJETIVO:
                severity = ""
                score = ""
                vector = ""
                exploit_score = ""
                impact_score = ""

                for version in ["cvssMetricV31", "cvssMetricV30", "cvssMetricV2"]:
                    if version in metrics:
                        m = metrics[version][0]
                        cvss = m.get("cvssData", {})
                        severity = m.get("baseSeverity", "")
                        score = cvss.get("baseScore", "")
                        vector = cvss.get("vectorString", "")
                        exploit_score = m.get("exploitabilityScore", "")
                        impact_score = m.get("impactScore", "")
                        break

                resultados_filtrados.append({
                    "cve_id": cve["id"],
                    "published": pub_date,
                    "last_modified": mod_date,
                    "source_identifier": source,
                    "description": description,
                    "severity": severity,
                    "cvss_score": score,
                    "vector": vector,
                    "exploitability_score": exploit_score,
                    "impact_score": impact_score
                })

        except Exception as e:
            print(f" Error procesando una entrada: {e}")

    start_index += page_size


## Importando los datos para la prepararcion de analizis

In [12]:
# Exportar a CSV
if resultados_filtrados:
    df = pd.DataFrame(resultados_filtrados)
    df.to_csv(ARCHIVO_SALIDA, index=False)
    print(f" Se guardaron {len(df)} vulnerabilidades en '{ARCHIVO_SALIDA}'.")
else:
    print(" No se encontraron vulnerabilidades para exportar.")

 Se guardaron 6071 vulnerabilidades en 'vulnerabilidades_2024.csv'.


# Vamos al 02_preprocesamiento_y_visualizaciones.ipynb para el tratamiento de datos